In [1]:
# 4/9/23
# KM Altenburger

# steps for downloading repo internally; run w/ "pytorch kernel"
# sudo feature install ttls_fwdproxy
# mgt import --src-type pypi sdist
# mgt import --src-type pypi nvidia-cublas-cu11==11.10.3.66
# mgt import --src-type pypi ogb

import sys
sys.path.insert(0, "/home/kaltenburger/fbsource/ogb")
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import ogb
import pandas as pd
import torch

# import torch_geometric
from ogb.nodeproppred import NodePropPredDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score
from sklearn.model_selection import train_test_split


Bad key "axes.color_cycle" on line 214 in
/home/kaltenburger/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


I0513 084137.673 _utils_internal.py:199] NCCL_DEBUG env var is set to None


I0513 084137.675 _utils_internal.py:217] NCCL_DEBUG is forced to WARN from None


### read & set-up dataset


In [2]:
dataset = NodePropPredDataset(name='ogbn-proteins')
g = dataset.graph # get edgelist and graph metadata
y = dataset.labels # outcomes
node_species_df = pd.DataFrame({'node_id':range(len(g['node_species'].flatten())), 'node_species': g['node_species'].flatten()})
node_species_df.head()

,node_id,node_species
0,0,3702
1,1,3702
2,2,3702
3,3,3702
4,4,3702


In [3]:
# create edgelist
edge_list_df = pd.DataFrame(g["edge_index"].T)
edge_list_df.columns = ['src','dst']
edge_list_df.head()

,src,dst
0,1,96401
1,96401,1
2,1,68343
3,68343,1
4,1,125069


In [4]:
# sanity check on inter-species connections
edge_list_df_check = edge_list_df.merge(node_species_df, how = 'left', left_on = 'src', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
edge_list_df_check = edge_list_df_check.merge(node_species_df, how = 'left', left_on = 'dst', right_on = 'node_id')
edge_list_df_check.drop('node_id', axis = 1, inplace=True)
#edge_list_df_check.head()
print(np.mean(edge_list_df_check['node_species_x']==edge_list_df_check['node_species_y']))
print(np.mean(edge_list_df_check['node_species_x']!=edge_list_df_check['node_species_y']))

0.8059610954678583


0.1940389045321417


In [5]:
y0 = pd.DataFrame(y)
y0 = y0.iloc[:,0:25] # part I
#y0 = y0.iloc[:,25:50] # part II
#y0 = y0.iloc[:,50:112] # part II
y0['node_id'] = range(len(y0[24])) # change here for each part
    
split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

X_train = y0['node_id'].iloc[train_idx]
y_train = y0.iloc[train_idx]

X_valid = y0['node_id'].iloc[valid_idx]
X_test = y0['node_id'].iloc[test_idx]
y_valid = y0.iloc[valid_idx]
y_test = y0.iloc[test_idx]



In [6]:
edge_w = pd.DataFrame(g['edge_feat'])
edge_w = edge_w.add_suffix('_weight')
edge_list_df = pd.concat([edge_list_df, edge_w], axis=1)
#edge_list_df.head()
# w0 = edge_w[j]
# edge_list_df['weight'] = w0
# edge_list_df.head()
# # merge in labels for training nodes on a user's connections
relevant_edges_tmp = edge_list_df.merge(y_train, left_on = 'dst', right_on='node_id', how = 'left')
relevant_edges_tmp.head()

,src,dst,0_weight,1_weight,2_weight,3_weight,4_weight,5_weight,6_weight,7_weight,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,node_id
0,1,96401,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,96401,1,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,68343,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68343.0
3,68343,1,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,125069,0.501,0.001,0.001,0.001,0.001,0.001,0.001,0.001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
intervals_v1 = relevant_edges_tmp[['0_weight','1_weight', '2_weight', '3_weight','6_weight','4_weight', '5_weight', '7_weight']].quantile([0.75])

In [8]:
np.array(intervals_v1.iloc[0])

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.091, 0.115, 0.177])

In [10]:
from sklearn.preprocessing import Binarizer
k=0
for j in range(8):
    #print(j)
    transformer = Binarizer().fit(relevant_edges_tmp[[str(j)+'_weight']])
    relevant_edges_tmp['bin_'+str(k)] = pd.DataFrame(transformer.transform((relevant_edges_tmp[[str(j)+'_weight']]<=np.array(intervals_v1.iloc[0])[j])+0).flatten())
    #print(relevant_edges_tmp.head())
    k=k+1
    relevant_edges_tmp['bin_'+str(k)] = pd.DataFrame(transformer.transform((relevant_edges_tmp[[str(j)+'_weight']]>np.array(intervals_v1.iloc[0])[j])+0).flatten())
    k=k+1

In [11]:
tmp = pd.DataFrame(relevant_edges_tmp[['bin_0',    'bin_1',    'bin_2',    'bin_3',    'bin_4',    'bin_5',
        'bin_6',    'bin_7',    'bin_8',    'bin_9',   'bin_10',   'bin_11',
        'bin_12',   'bin_13', 'bin_14',   'bin_15' ]].drop_duplicates())

In [12]:
tmp.reset_index(inplace=True,drop=True)
tmp.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,bin_5,bin_6,bin_7,bin_8,bin_9,bin_10,bin_11,bin_12,bin_13,bin_14,bin_15
0,0,1,1,0,1,0,1,0,1,0,1,0,1,0,1,0
1,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,0
2,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1
3,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0
4,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0


In [13]:
tmp['edge_bin'] = 0
tmp['edge_bin'] = pd.DataFrame(range(np.shape(tmp)[0]))

In [14]:
relevant_edges_tmp['edge_bin']=0
for k in np.array(tmp.edge_bin):
    #print(k)
    relevant_edges_tmp['edge_bin'][(relevant_edges_tmp.bin_0==tmp.bin_0[k]) & \
(relevant_edges_tmp.bin_1==tmp.bin_1[k]) & \
(relevant_edges_tmp.bin_2==tmp.bin_2[k]) & \
(relevant_edges_tmp.bin_3==tmp.bin_3[k]) & \
(relevant_edges_tmp.bin_4==tmp.bin_4[k]) & \
(relevant_edges_tmp.bin_5==tmp.bin_5[k]) & \
(relevant_edges_tmp.bin_6==tmp.bin_6[k]) & \
(relevant_edges_tmp.bin_7==tmp.bin_7[k]) & \
(relevant_edges_tmp.bin_8==tmp.bin_8[k]) & \
(relevant_edges_tmp.bin_9==tmp.bin_9[k]) & \
(relevant_edges_tmp.bin_10==tmp.bin_10[k]) & \
(relevant_edges_tmp.bin_11==tmp.bin_11[k]) & \
(relevant_edges_tmp.bin_12==tmp.bin_12[k]) & \
(relevant_edges_tmp.bin_13==tmp.bin_13[k]) & \
(relevant_edges_tmp.bin_14==tmp.bin_14[k]) & \
(relevant_edges_tmp.bin_15==tmp.bin_15[k]) ] = tmp.edge_bin[k]

<ipython-input-14-4b1b563fe631>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
from sklearn.ensemble import RandomForestClassifier

# set-up loop
rocauc_v2_ud_w = []


for i in range(0,25): # loop through outcomes part 1
#for i in range(25,50):
#for i in range(50,112):
    print(i)

    relevant_edges = relevant_edges_tmp[["src", "dst", "node_id", i, 'edge_bin']]
    relevant_edges.columns = ["src", "dst",  "node_id", "y", 'edge_bin']
    label1 = relevant_edges.groupby(["src", 'edge_bin'])["y"].sum().reset_index() # same by edgebin
    labeltotal = relevant_edges.groupby(["src", 'edge_bin'])["y"].count().reset_index()
    relevant_edgelist = labeltotal.merge(label1, on=["src", 'edge_bin'])

    ## this is already by edge bin
    relevant_edgelist["label_0"] = relevant_edgelist["y_x"] - relevant_edgelist["y_y"]
    relevant_edgelist.rename(columns={"y_y": "label_1"}, inplace=True)
    relevant_edgelist.drop(["y_x"], axis=1, inplace=True)

    ## v0 - linear combination of single layers
    normalizer = pd.DataFrame(y_train[i].value_counts(normalize=True, ascending=True))

    # step 1 - one hop scores
    step1_one_hop_scores = pd.DataFrame(
        {
            "src": relevant_edgelist.src,
            "edge_bin": relevant_edgelist.edge_bin,
            "frac": np.log(
                (relevant_edgelist["label_1"] + 1) / (relevant_edgelist["label_0"] + 1)
            )
            * list((normalizer.iloc[0]) / (normalizer.iloc[1]))[0],
        }
    )

    two_hop_stg_v2 = relevant_edges.merge(
            step1_one_hop_scores, left_on=["dst","edge_bin"], right_on=["src","edge_bin"], how="inner" # keep edge_bin and one-hop scores same
        )

    two_hop_stg = two_hop_stg_v2.groupby(['src_x','edge_bin'])['frac'].sum().reset_index()
    two_hop_v2 = pd.pivot_table(two_hop_stg, values = 'frac', columns = 'edge_bin', index = 'src_x').reset_index().fillna(0)
    two_hop_v2 = two_hop_v2.add_prefix('score_')

    two_hop_v2_valid = two_hop_v2.merge(
            y_valid, left_on="score_src_x", right_on="node_id", how="right"
        )
    clf = LogisticRegression(random_state=0).fit(
        two_hop_v2_valid[['score_0', 'score_1', 'score_2', 'score_3',
       'score_4', 'score_5', 'score_6', 'score_7', 'score_8', 'score_9',
       'score_10', 'score_11', 'score_12', 'score_13', 'score_14',
       'score_15', 'score_16', 'score_17', 'score_18', 'score_19',
       'score_20', 'score_21', 'score_22', 'score_23', 'score_24',
       'score_25', 'score_26', 'score_27', 'score_28', 'score_29',
       'score_30', 'score_31', 'score_32', 'score_33', 'score_34',
       'score_35', 'score_36', 'score_37', 'score_38', 'score_39',
       'score_40', 'score_41', 'score_42', 'score_43', 'score_44',
       'score_45', 'score_46', 'score_47', 'score_48', 'score_49',
       'score_50', 'score_51', 'score_52', 'score_53', 'score_54',
       'score_55', 'score_56', 'score_57', 'score_58', 'score_59',
       'score_60', 'score_61', 'score_62', 'score_63', 'score_64',
       'score_65', 'score_66', 'score_67', 'score_68', 'score_69',
       'score_70', 'score_71', 'score_72', 'score_73', 'score_74',
       'score_75', 'score_76', 'score_77', 'score_78', 'score_79',
       'score_80', 'score_81', 'score_82', 'score_83', 'score_84',
       'score_85', 'score_86', 'score_87', 'score_88', 'score_89',
       'score_90', 'score_91', 'score_92', 'score_93', 'score_94',
       'score_95', 'score_96', 'score_97', 'score_98', 'score_99',
       'score_100', 'score_101', 'score_102', 'score_103', 'score_104',
       'score_105', 'score_106', 'score_107', 'score_108', 'score_109',
       'score_110', 'score_111', 'score_112', 'score_113', 'score_114',
       'score_115', 'score_116', 'score_117', 'score_118', 'score_119',
       'score_120', 'score_121', 'score_122', 'score_123', 'score_124',
       'score_125', 'score_126', 'score_127']], two_hop_v2_valid[i]
    )


    pred_v2 = y_test.merge(two_hop_v2, right_on="score_src_x", left_on="node_id", how="left")
    print(
        "roc-auc, v1: ",
        roc_auc_score(
            pred_v2[i],
            clf.predict_proba(pred_v2[['score_0', 'score_1', 'score_2', 'score_3',
       'score_4', 'score_5', 'score_6', 'score_7', 'score_8', 'score_9',
       'score_10', 'score_11', 'score_12', 'score_13', 'score_14',
       'score_15', 'score_16', 'score_17', 'score_18', 'score_19',
       'score_20', 'score_21', 'score_22', 'score_23', 'score_24',
       'score_25', 'score_26', 'score_27', 'score_28', 'score_29',
       'score_30', 'score_31', 'score_32', 'score_33', 'score_34',
       'score_35', 'score_36', 'score_37', 'score_38', 'score_39',
       'score_40', 'score_41', 'score_42', 'score_43', 'score_44',
       'score_45', 'score_46', 'score_47', 'score_48', 'score_49',
       'score_50', 'score_51', 'score_52', 'score_53', 'score_54',
       'score_55', 'score_56', 'score_57', 'score_58', 'score_59',
       'score_60', 'score_61', 'score_62', 'score_63', 'score_64',
       'score_65', 'score_66', 'score_67', 'score_68', 'score_69',
       'score_70', 'score_71', 'score_72', 'score_73', 'score_74',
       'score_75', 'score_76', 'score_77', 'score_78', 'score_79',
       'score_80', 'score_81', 'score_82', 'score_83', 'score_84',
       'score_85', 'score_86', 'score_87', 'score_88', 'score_89',
       'score_90', 'score_91', 'score_92', 'score_93', 'score_94',
       'score_95', 'score_96', 'score_97', 'score_98', 'score_99',
       'score_100', 'score_101', 'score_102', 'score_103', 'score_104',
       'score_105', 'score_106', 'score_107', 'score_108', 'score_109',
       'score_110', 'score_111', 'score_112', 'score_113', 'score_114',
       'score_115', 'score_116', 'score_117', 'score_118', 'score_119',
       'score_120', 'score_121', 'score_122', 'score_123', 'score_124',
       'score_125', 'score_126', 'score_127']])[:,1]
        ),
    )
    rocauc_v2_ud_w.append(roc_auc_score(
            pred_v2[i],
            clf.predict_proba(pred_v2[['score_0', 'score_1', 'score_2', 'score_3',
       'score_4', 'score_5', 'score_6', 'score_7', 'score_8', 'score_9',
       'score_10', 'score_11', 'score_12', 'score_13', 'score_14',
       'score_15', 'score_16', 'score_17', 'score_18', 'score_19',
       'score_20', 'score_21', 'score_22', 'score_23', 'score_24',
       'score_25', 'score_26', 'score_27', 'score_28', 'score_29',
       'score_30', 'score_31', 'score_32', 'score_33', 'score_34',
       'score_35', 'score_36', 'score_37', 'score_38', 'score_39',
       'score_40', 'score_41', 'score_42', 'score_43', 'score_44',
       'score_45', 'score_46', 'score_47', 'score_48', 'score_49',
       'score_50', 'score_51', 'score_52', 'score_53', 'score_54',
       'score_55', 'score_56', 'score_57', 'score_58', 'score_59',
       'score_60', 'score_61', 'score_62', 'score_63', 'score_64',
       'score_65', 'score_66', 'score_67', 'score_68', 'score_69',
       'score_70', 'score_71', 'score_72', 'score_73', 'score_74',
       'score_75', 'score_76', 'score_77', 'score_78', 'score_79',
       'score_80', 'score_81', 'score_82', 'score_83', 'score_84',
       'score_85', 'score_86', 'score_87', 'score_88', 'score_89',
       'score_90', 'score_91', 'score_92', 'score_93', 'score_94',
       'score_95', 'score_96', 'score_97', 'score_98', 'score_99',
       'score_100', 'score_101', 'score_102', 'score_103', 'score_104',
       'score_105', 'score_106', 'score_107', 'score_108', 'score_109',
       'score_110', 'score_111', 'score_112', 'score_113', 'score_114',
       'score_115', 'score_116', 'score_117', 'score_118', 'score_119',
       'score_120', 'score_121', 'score_122', 'score_123', 'score_124',
       'score_125', 'score_126', 'score_127']])[:,1]
        ))


0


roc-auc, v1:  0.4722436779047938
1


roc-auc, v1:  0.552788929384319
2


roc-auc, v1:  0.6431188273226476
3


roc-auc, v1:  0.697286455791277
4


...

roc-auc, v1:  0.6980471935107773
21


roc-auc, v1:  0.7797823106975246
22


roc-auc, v1:  0.7705034846542953
23


roc-auc, v1:  0.7715661920938138
24


roc-auc, v1:  0.7740138873087041


In [16]:
print(np.mean(rocauc_v2_ud_w))


0.721155537829489


In [23]:
with open('rocauc_v2_ud_w_part1_multilayer_original.npy', 'wb') as f:
#with open('rocauc_v2_ud_w_part2_multilayer_original.npy', 'wb') as f:
#with open('rocauc_v2_ud_w_part3_multilayer_original.npy', 'wb') as f:
    np.save(f, rocauc_v2_ud_w)